# Code from previous lessons

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold

In [2]:
df = pd.read_csv('../../churn-prediction/customers_data.csv')

In [3]:
df.columns = df.columns.str.lower().str.replace(" ", "_")

str_cols = list(df.dtypes[df.dtypes == 'object'].index)

for col in str_cols:
    df[col] = df[col].str.lower().str.replace(" ", "_")
    
df.totalcharges = pd.to_numeric(df.totalcharges, errors='coerce')

df.totalcharges = df.totalcharges.fillna(0)

df.churn = (df.churn == 'yes').astype(int)

In [4]:
full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_valid = train_test_split(full_train, test_size=0.25, random_state=1)

In [5]:
y_test = df_test.churn.values
y_valid = df_valid.churn.values
y_train = df_train.churn.values

df_train = df_train.reset_index(drop=True)
df_valid = df_valid.reset_index(drop=True)
df_train = df_train.reset_index(drop=True)

del df_test['churn']
del df_valid['churn']
del df_train['churn']

In [6]:
numerical = ['tenure', 'monthlycharges', 'totalcharges']

categorical = ['gender', 'seniorcitizen', 'partner', 'dependents',
               'phoneservice', 'multiplelines', 'internetservice',
               'onlinesecurity', 'onlinebackup', 'deviceprotection',
               'techsupport', 'streamingtv', 'streamingmovies',
               'contract', 'paperlessbilling', 'paymentmethod']

In [7]:
def train(df, y_train, C):
    dicts = df[numerical + categorical].to_dict(orient='records')
    
    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(dicts)
    
    model = LogisticRegression(solver='liblinear', dual=False, max_iter=2000,
                                C=C, random_state=42)
    model.fit(X_train, y_train)
    
    return dv, model

In [8]:
def predict(df, dv, model):
    valid = df[numerical + categorical].to_dict(orient='records')
    X = dv.transform(valid)
    y_pred = model.predict_proba(X)[:, 1]
    
    return y_pred

In [16]:
kfold = KFold(n_splits=5, shuffle=True, random_state=1)
C = 1.0

In [18]:
scores = []

for trainIdx, validIdx in kfold.split(full_train):
    df_train = full_train.iloc[trainIdx]
    df_valid = full_train.iloc[validIdx]
    
    y_train = df_train.churn.values
    y_valid = df_valid.churn.values
    
    dv, model = train(df_train, y_train, C=C)
    y_pred = predict(df_valid, dv, model)
    
    auc = roc_auc_score(y_valid, y_pred)
    scores.append(auc)

In [20]:
np.mean(scores), np.std(scores)

(0.8411783377274789, 0.007388401938629943)

In [11]:
dv, model = train(full_train, full_train.churn.values, C=1.0)
y_pred = predict(df_test, dv, model)

auc = roc_auc_score(y_test, y_pred)

In [12]:
auc

0.8579400803839363

# Saving the model

In [21]:
import pickle

In [22]:
with open(f'model_C={C}.bin', 'wb') as churn_model:
    pickle.dump((dv, model), churn_model)

In [23]:
with open('model_C=1.0.bin', 'rb') as churn_res:
    dv, model = pickle.load(churn_res)

In [26]:
dv, model

(DictVectorizer(sparse=False),
 LogisticRegression(max_iter=2000, random_state=42, solver='liblinear'))

In [6]:
customer = {
    "customerid": "8879-zkjof",
    "gender": "female",
    "seniorcitizen": 1,
    "partner": "no",
    "dependents": "no",
    "tenure": 41,
    "phoneservice": "yes",
    "multiplelines": "no",
    "internetservice": "dsl",
    "onlinesecurity": 'yes',
    "onlinebackup": "no",
    "deviceprotection": "yes",
    "techsupport": "yes",
    "streamingtv": "yes",
    "streamingmovies": "yes",
    "contract": "one_year",
    "paperlessbilling": "yes",
    "paymentmethod": "bank_transfer_(automatic)",
    "monthlycharges": 79.85,
    "totalcharges": 3320.75,
}

In [35]:
X = dv.transform([customer])

In [36]:
model.predict_proba(X)[0, 1]

0.09069205179393283

# Using the service

In [11]:
import requests

In [12]:
url = 'http://192.168.0.101:7000/predict'
response = requests.post(url, json=customer)
result = response.json()

In [13]:
result

{'churn': False, 'pred.': 0.09069205179393283}